In [1]:
# import dependencies
import pandas as pd
import numpy as np
import os
import requests
import time
import scipy
import calendar

In [38]:
# read in 2020 monthly CSV's
Jan2020_df=pd.read_csv("2020/202001-citibike-tripdata.csv", encoding="utf-8")
Feb2020_df=pd.read_csv("2020/202002-citibike-tripdata.csv", encoding="utf-8")
Mar2020_df=pd.read_csv("2020/202003-citibike-tripdata.csv", encoding="utf-8")
Apr2020_df=pd.read_csv("2020/202004-citibike-tripdata.csv", encoding="utf-8")
May2020_df=pd.read_csv("2020/202005-citibike-tripdata.csv", encoding="utf-8")
Jun2020_df=pd.read_csv("2020/202006-citibike-tripdata.csv", encoding="utf-8")
Jul2020_df=pd.read_csv("2020/202007-citibike-tripdata.csv", encoding="utf-8")
Aug2020_df=pd.read_csv("2020/202008-citibike-tripdata.csv", encoding="utf-8")
Sept2020_df=pd.read_csv("2020/202009-citibike-tripdata.csv", encoding="utf-8")
Oct2020_df=pd.read_csv("2020/202010-citibike-tripdata.csv", encoding="utf-8")

# Jan 2020 data

In [42]:
# convert start time to date tim format and split into two columns, date and time
d = pd.to_datetime(Jan2020_df['starttime'])
Jan2020_df['date'] = d.dt.date
Jan2020_df['time'] = d.dt.time

In [45]:
# convert create month column and convert to month name
Jan2020_df['starttimemonth'] = pd.DatetimeIndex(Jan2020_df['date']).month
Jan2020_df['starttimemonth'] = Jan2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Jan2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1,2020-01-01,00:00:55.390000,Jan
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1,2020-01-01,00:01:08.102000,Jan
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1,2020-01-01,00:01:42.140000,Jan
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1,2020-01-01,00:01:45.561000,Jan
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,1,2020-01-01,00:01:45.788000,Jan


In [46]:
# drop unnecessary columns 
clean_Jan2020_df = Jan2020_df.drop(columns = ["birth year", "gender","time"])
clean_Jan2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,2020-01-01,Jan
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,2020-01-01,Jan
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,2020-01-01,Jan
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,2020-01-01,Jan
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,2020-01-01,Jan


In [47]:
# calculate summary fields
# total trips per month
Trips_Jan = clean_Jan2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Jan = clean_Jan2020_df.loc[clean_Jan2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Jan = Subtrips_Jan/Trips_Jan*100
Custtrips_Jan = clean_Jan2020_df.loc[clean_Jan2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Jan = Custtrips_Jan/Trips_Jan*100
# calculate total stations ids
start_station_id_count_Jan = clean_Jan2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Jan=clean_Jan2020_df["bikeid"].nunique()
bike_id_count_Jan

13049

In [48]:
# create monthly summary
Jan_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Jan],
                     "Subscriber Trips Count":[Subtrips_Jan],
                     "% Subscriber Trips":[percent_subtrips_Jan],
                     "Customer Trips Count":[Custtrips_Jan],
                     "% Customer Trips":[percent_custtrips_Jan],
                     "Start Station Count":[start_station_id_count_Jan],
                     "Bike ID Count":[bike_id_count_Jan]})
Jan_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,1240596,1145819,92.360366,94777,7.639634,893,13049


In [49]:
# add month column to summary
Jan_2020_Summary['Month'] = clean_Jan2020_df['starttimemonth']
Jan_2020_Summary=Jan_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Jan_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Jan,1240596,1145819,92.360366,94777,7.639634,893,13049


# Feb Data

In [50]:
d = pd.to_datetime(Feb2020_df['starttime'])
Feb2020_df['date'] = d.dt.date
Feb2020_df['time'] = d.dt.time

In [51]:
# convert create month column and convert to month name
Feb2020_df['starttimemonth'] = pd.DatetimeIndex(Feb2020_df['date']).month
Feb2020_df['starttimemonth'] = Feb2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Feb2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,1404,2020-02-01 00:00:05.9460,2020-02-01 00:23:30.7240,316,Fulton St & William St,40.709560,-74.006536,481,S 3 St & Bedford Ave,40.712605,-73.962644,28874,Customer,1995,1,2020-02-01,00:00:05.946000,Feb
1,1301,2020-02-01 00:00:06.2230,2020-02-01 00:21:48.0580,237,E 11 St & 2 Ave,40.730473,-73.986724,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,32588,Subscriber,1991,1,2020-02-01,00:00:06.223000,Feb
2,474,2020-02-01 00:00:15.7210,2020-02-01 00:08:10.3440,528,2 Ave & E 31 St,40.742909,-73.977061,3785,W 42 St & 6 Ave,40.754920,-73.984550,41013,Subscriber,1994,1,2020-02-01,00:00:15.721000,Feb
3,487,2020-02-01 00:00:21.0520,2020-02-01 00:08:28.7520,380,W 4 St & 7 Ave S,40.734011,-74.002939,3263,Cooper Square & Astor Pl,40.729515,-73.990753,27581,Subscriber,1973,2,2020-02-01,00:00:21.052000,Feb
4,619,2020-02-01 00:00:27.4000,2020-02-01 00:10:47.0640,472,E 32 St & Park Ave,40.745712,-73.981948,237,E 11 St & 2 Ave,40.730473,-73.986724,29062,Subscriber,1994,1,2020-02-01,00:00:27.400000,Feb


In [52]:
# drop unnecessary columns 
clean_Feb2020_df = Feb2020_df.drop(columns = ["birth year", "gender","time"])
clean_Feb2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,1404,2020-02-01 00:00:05.9460,2020-02-01 00:23:30.7240,316,Fulton St & William St,40.709560,-74.006536,481,S 3 St & Bedford Ave,40.712605,-73.962644,28874,Customer,2020-02-01,Feb
1,1301,2020-02-01 00:00:06.2230,2020-02-01 00:21:48.0580,237,E 11 St & 2 Ave,40.730473,-73.986724,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,32588,Subscriber,2020-02-01,Feb
2,474,2020-02-01 00:00:15.7210,2020-02-01 00:08:10.3440,528,2 Ave & E 31 St,40.742909,-73.977061,3785,W 42 St & 6 Ave,40.754920,-73.984550,41013,Subscriber,2020-02-01,Feb
3,487,2020-02-01 00:00:21.0520,2020-02-01 00:08:28.7520,380,W 4 St & 7 Ave S,40.734011,-74.002939,3263,Cooper Square & Astor Pl,40.729515,-73.990753,27581,Subscriber,2020-02-01,Feb
4,619,2020-02-01 00:00:27.4000,2020-02-01 00:10:47.0640,472,E 32 St & Park Ave,40.745712,-73.981948,237,E 11 St & 2 Ave,40.730473,-73.986724,29062,Subscriber,2020-02-01,Feb


In [53]:
# calculate summary fields
# total trips per month
Trips_Feb = clean_Feb2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Feb = clean_Feb2020_df.loc[clean_Feb2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Feb = Subtrips_Feb/Trips_Feb*100
Custtrips_Feb = clean_Feb2020_df.loc[clean_Feb2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Feb = Custtrips_Feb/Trips_Feb*100
# calculate total stations ids
start_station_id_count_Feb = clean_Feb2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Feb=clean_Feb2020_df["bikeid"].nunique()
bike_id_count_Feb

13218

In [54]:
# create monthly summary
Feb_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Feb],
                     "Subscriber Trips Count":[Subtrips_Feb],
                     "% Subscriber Trips":[percent_subtrips_Feb],
                     "Customer Trips Count":[Custtrips_Feb],
                     "% Customer Trips":[percent_custtrips_Feb],
                     "Start Station Count":[start_station_id_count_Feb],
                     "Bike ID Count":[bike_id_count_Feb]})
Feb_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,1146830,1054927,91.986345,91903,8.013655,890,13218


In [55]:
# add month column to summary
Feb_2020_Summary['Month'] = clean_Feb2020_df['starttimemonth']
Feb_2020_Summary=Feb_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Feb_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Feb,1146830,1054927,91.986345,91903,8.013655,890,13218


# March Data

In [57]:
d = pd.to_datetime(Mar2020_df['starttime'])
Mar2020_df['date'] = d.dt.date
Mar2020_df['time'] = d.dt.time

In [59]:
# convert create month column and convert to month name
Mar2020_df['starttimemonth'] = pd.DatetimeIndex(Mar2020_df['date']).month
Mar2020_df['starttimemonth'] = Mar2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Mar2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,1589,2020-03-01 00:00:03.6400,2020-03-01 00:26:32.9860,224,Spruce St & Nassau St,40.711464,-74.005524,3574,Prospect Pl & Underhill Ave,40.676969,-73.965790,16214,Subscriber,1980,1,2020-03-01,00:00:03.640000,Mar
1,389,2020-03-01 00:00:16.7560,2020-03-01 00:06:46.0620,293,Lafayette St & E 8 St,40.730207,-73.991026,223,W 13 St & 7 Ave,40.737815,-73.999947,29994,Subscriber,1991,2,2020-03-01,00:00:16.756000,Mar
2,614,2020-03-01 00:00:20.0580,2020-03-01 00:10:34.2200,379,W 31 St & 7 Ave,40.749156,-73.991600,515,W 43 St & 10 Ave,40.760094,-73.994618,39853,Subscriber,1991,1,2020-03-01,00:00:20.058000,Mar
3,597,2020-03-01 00:00:24.3510,2020-03-01 00:10:22.3390,3739,Perry St & Greenwich Ave,40.735918,-74.000939,325,E 19 St & 3 Ave,40.736245,-73.984738,42608,Subscriber,1989,1,2020-03-01,00:00:24.351000,Mar
4,1920,2020-03-01 00:00:26.1120,2020-03-01 00:32:26.2680,236,St Marks Pl & 2 Ave,40.728419,-73.987140,3124,46 Ave & 5 St,40.747310,-73.954510,36288,Subscriber,1993,1,2020-03-01,00:00:26.112000,Mar


In [60]:
# drop unnecessary columns 
clean_Mar2020_df = Mar2020_df.drop(columns = ["birth year", "gender","time"])
clean_Mar2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,1589,2020-03-01 00:00:03.6400,2020-03-01 00:26:32.9860,224,Spruce St & Nassau St,40.711464,-74.005524,3574,Prospect Pl & Underhill Ave,40.676969,-73.965790,16214,Subscriber,2020-03-01,Mar
1,389,2020-03-01 00:00:16.7560,2020-03-01 00:06:46.0620,293,Lafayette St & E 8 St,40.730207,-73.991026,223,W 13 St & 7 Ave,40.737815,-73.999947,29994,Subscriber,2020-03-01,Mar
2,614,2020-03-01 00:00:20.0580,2020-03-01 00:10:34.2200,379,W 31 St & 7 Ave,40.749156,-73.991600,515,W 43 St & 10 Ave,40.760094,-73.994618,39853,Subscriber,2020-03-01,Mar
3,597,2020-03-01 00:00:24.3510,2020-03-01 00:10:22.3390,3739,Perry St & Greenwich Ave,40.735918,-74.000939,325,E 19 St & 3 Ave,40.736245,-73.984738,42608,Subscriber,2020-03-01,Mar
4,1920,2020-03-01 00:00:26.1120,2020-03-01 00:32:26.2680,236,St Marks Pl & 2 Ave,40.728419,-73.987140,3124,46 Ave & 5 St,40.747310,-73.954510,36288,Subscriber,2020-03-01,Mar


In [61]:
# calculate summary fields
# total trips per month
Trips_Mar = clean_Mar2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Mar = clean_Mar2020_df.loc[clean_Mar2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Mar = Subtrips_Mar/Trips_Mar*100
Custtrips_Mar = clean_Mar2020_df.loc[clean_Mar2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Mar = Custtrips_Mar/Trips_Mar*100
# calculate total stations ids
start_station_id_count_Mar = clean_Mar2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Mar=clean_Mar2020_df["bikeid"].nunique()
bike_id_count_Mar

14882

In [62]:
# create monthly summary
Mar_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Mar],
                     "Subscriber Trips Count":[Subtrips_Mar],
                     "% Subscriber Trips":[percent_subtrips_Mar],
                     "Customer Trips Count":[Custtrips_Mar],
                     "% Customer Trips":[percent_custtrips_Mar],
                     "Start Station Count":[start_station_id_count_Mar],
                     "Bike ID Count":[bike_id_count_Mar]})
Mar_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,1068457,914068,85.550284,154389,14.449716,889,14882


In [63]:
# add month column to summary
Mar_2020_Summary['Month'] = clean_Mar2020_df['starttimemonth']
Mar_2020_Summary=Mar_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Mar_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Mar,1068457,914068,85.550284,154389,14.449716,889,14882


# April Data

In [64]:
d = pd.to_datetime(Apr2020_df['starttime'])
Apr2020_df['date'] = d.dt.date
Apr2020_df['time'] = d.dt.time

In [65]:
# convert create month column and convert to month name
Apr2020_df['starttimemonth'] = pd.DatetimeIndex(Apr2020_df['date']).month
Apr2020_df['starttimemonth'] = Apr2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Apr2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,534,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,3656,E 2 St & Avenue A,40.723077,-73.985836,545,E 23 St & 1 Ave,40.736502,-73.978095,15162,Subscriber,1990,2,2020-04-01,00:00:15.280000,Apr
1,529,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,3163,Central Park West & W 68 St,40.773407,-73.977825,3301,Columbus Ave & W 95 St,40.791956,-73.968087,42861,Subscriber,1974,1,2020-04-01,00:02:28.943000,Apr
2,1488,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,3164,Columbus Ave & W 72 St,40.777057,-73.978985,3605,31 Ave & Crescent St,40.765835,-73.926547,35989,Subscriber,1982,1,2020-04-01,00:02:56.090000,Apr
3,341,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,3827,Halsey St & Broadway,40.685650,-73.915640,3869,Van Buren St & Broadway,40.692000,-73.926170,37548,Subscriber,1996,1,2020-04-01,00:03:25.141000,Apr
4,2206,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,3890,Grove St & Broadway,40.690470,-73.923511,3058,Lewis Ave & Kosciuszko St,40.692371,-73.937054,39701,Subscriber,1977,1,2020-04-01,00:05:35.279000,Apr


In [66]:
# drop unnecessary columns 
clean_Apr2020_df = Apr2020_df.drop(columns = ["birth year", "gender","time"])
clean_Apr2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,534,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,3656,E 2 St & Avenue A,40.723077,-73.985836,545,E 23 St & 1 Ave,40.736502,-73.978095,15162,Subscriber,2020-04-01,Apr
1,529,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,3163,Central Park West & W 68 St,40.773407,-73.977825,3301,Columbus Ave & W 95 St,40.791956,-73.968087,42861,Subscriber,2020-04-01,Apr
2,1488,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,3164,Columbus Ave & W 72 St,40.777057,-73.978985,3605,31 Ave & Crescent St,40.765835,-73.926547,35989,Subscriber,2020-04-01,Apr
3,341,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,3827,Halsey St & Broadway,40.685650,-73.915640,3869,Van Buren St & Broadway,40.692000,-73.926170,37548,Subscriber,2020-04-01,Apr
4,2206,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,3890,Grove St & Broadway,40.690470,-73.923511,3058,Lewis Ave & Kosciuszko St,40.692371,-73.937054,39701,Subscriber,2020-04-01,Apr


In [67]:
# calculate summary fields
# total trips per month
Trips_Apr = clean_Apr2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Apr = clean_Apr2020_df.loc[clean_Apr2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Apr = Subtrips_Apr/Trips_Apr*100
Custtrips_Apr = clean_Apr2020_df.loc[clean_Apr2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Apr = Custtrips_Apr/Trips_Apr*100
# calculate total stations ids
start_station_id_count_Apr = clean_Apr2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Apr=clean_Apr2020_df["bikeid"].nunique()
bike_id_count_Apr

14780

In [68]:
# create monthly summary
Apr_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Apr],
                     "Subscriber Trips Count":[Subtrips_Apr],
                     "% Subscriber Trips":[percent_subtrips_Apr],
                     "Customer Trips Count":[Custtrips_Apr],
                     "% Customer Trips":[percent_custtrips_Apr],
                     "Start Station Count":[start_station_id_count_Apr],
                     "Bike ID Count":[bike_id_count_Apr]})
Apr_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,682762,516495,75.647883,166267,24.352117,887,14780


In [69]:
# add month column to summary
Apr_2020_Summary['Month'] = clean_Apr2020_df['starttimemonth']
Apr_2020_Summary=Apr_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Apr_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Apr,682762,516495,75.647883,166267,24.352117,887,14780


# May Data

In [70]:
d = pd.to_datetime(May2020_df['starttime'])
May2020_df['date'] = d.dt.date
May2020_df['time'] = d.dt.time

In [71]:
# convert create month column and convert to month name
May2020_df['starttimemonth'] = pd.DatetimeIndex(May2020_df['date']).month
May2020_df['starttimemonth'] = May2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
May2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,336,2020-05-01 00:01:06.3380,2020-05-01 00:06:42.9560,2017,E 43 St & 2 Ave,40.750224,-73.971214,465,Broadway & W 41 St,40.755136,-73.986580,18015,Subscriber,1993,1,2020-05-01,00:01:06.338000,May
1,853,2020-05-01 00:01:08.1950,2020-05-01 00:15:21.2150,3376,E 65 St & 2 Ave,40.764719,-73.962221,478,11 Ave & W 41 St,40.760301,-73.998842,35905,Subscriber,1989,1,2020-05-01,00:01:08.195000,May
2,78,2020-05-01 00:01:08.5260,2020-05-01 00:02:27.0490,396,Lefferts Pl & Franklin Ave,40.680342,-73.955769,3789,Fulton St & Irving Pl,40.681860,-73.959432,33350,Subscriber,1938,2,2020-05-01,00:01:08.526000,May
3,1185,2020-05-01 00:02:22.4380,2020-05-01 00:22:08.1220,3630,Frederick Douglass Blvd & W 115 St,40.803865,-73.955931,3506,Lexington Ave & E 120 St,40.801307,-73.939817,43790,Subscriber,1996,1,2020-05-01,00:02:22.438000,May
4,272,2020-05-01 00:02:32.8280,2020-05-01 00:07:05.0880,3776,Central Ave & Starr Street,40.700003,-73.928340,3068,Humboldt St & Varet St,40.703172,-73.940636,39912,Subscriber,1993,1,2020-05-01,00:02:32.828000,May


In [73]:
# drop unnecessary columns 
clean_May2020_df = May2020_df.drop(columns = ["birth year", "gender","time"])
clean_May2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,336,2020-05-01 00:01:06.3380,2020-05-01 00:06:42.9560,2017,E 43 St & 2 Ave,40.750224,-73.971214,465,Broadway & W 41 St,40.755136,-73.986580,18015,Subscriber,2020-05-01,May
1,853,2020-05-01 00:01:08.1950,2020-05-01 00:15:21.2150,3376,E 65 St & 2 Ave,40.764719,-73.962221,478,11 Ave & W 41 St,40.760301,-73.998842,35905,Subscriber,2020-05-01,May
2,78,2020-05-01 00:01:08.5260,2020-05-01 00:02:27.0490,396,Lefferts Pl & Franklin Ave,40.680342,-73.955769,3789,Fulton St & Irving Pl,40.681860,-73.959432,33350,Subscriber,2020-05-01,May
3,1185,2020-05-01 00:02:22.4380,2020-05-01 00:22:08.1220,3630,Frederick Douglass Blvd & W 115 St,40.803865,-73.955931,3506,Lexington Ave & E 120 St,40.801307,-73.939817,43790,Subscriber,2020-05-01,May
4,272,2020-05-01 00:02:32.8280,2020-05-01 00:07:05.0880,3776,Central Ave & Starr Street,40.700003,-73.928340,3068,Humboldt St & Varet St,40.703172,-73.940636,39912,Subscriber,2020-05-01,May


In [74]:
# calculate summary fields
# total trips per month
Trips_May = clean_May2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_May = clean_May2020_df.loc[clean_May2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_May = Subtrips_May/Trips_May*100
Custtrips_May = clean_May2020_df.loc[clean_May2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_May = Custtrips_May/Trips_May*100
# calculate total stations ids
start_station_id_count_May = clean_May2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_May=clean_May2020_df["bikeid"].nunique()
bike_id_count_May

15003

In [75]:
# create monthly summary
May_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_May],
                     "Subscriber Trips Count":[Subtrips_May],
                     "% Subscriber Trips":[percent_subtrips_May],
                     "Customer Trips Count":[Custtrips_May],
                     "% Customer Trips":[percent_custtrips_May],
                     "Start Station Count":[start_station_id_count_May],
                     "Bike ID Count":[bike_id_count_May]})
May_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,1487890,1012059,68.019746,475831,31.980254,923,15003


In [76]:
# add month column to summary
May_2020_Summary['Month'] = clean_May2020_df['starttimemonth']
May_2020_Summary=May_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
May_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,May,1487890,1012059,68.019746,475831,31.980254,923,15003


# June Data

In [77]:
d = pd.to_datetime(Jun2020_df['starttime'])
Jun2020_df['date'] = d.dt.date
Jun2020_df['time'] = d.dt.time

In [78]:
# convert create month column and convert to month name
Jun2020_df['starttimemonth'] = pd.DatetimeIndex(Jun2020_df['date']).month
Jun2020_df['starttimemonth'] = Jun2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Jun2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,1062,2020-06-01 00:00:03.3720,2020-06-01 00:17:46.2080,3419,Douglass St & 4 Ave,40.679279,-73.981540,3419,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2,2020-06-01,00:00:03.372000,Jun
1,3810,2020-06-01 00:00:03.5530,2020-06-01 01:03:33.9360,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0,2020-06-01,00:00:03.553000,Jun
2,1017,2020-06-01 00:00:09.6140,2020-06-01 00:17:06.8330,389,Broadway & Berry St,40.710446,-73.965251,3562,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2,2020-06-01,00:00:09.614000,Jun
3,226,2020-06-01 00:00:12.1780,2020-06-01 00:03:58.8640,3255,8 Ave & W 31 St,40.750585,-73.994685,505,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0,2020-06-01,00:00:12.178000,Jun
4,1437,2020-06-01 00:00:21.2550,2020-06-01 00:24:18.9650,367,E 53 St & Lexington Ave,40.758281,-73.970694,497,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2,2020-06-01,00:00:21.255000,Jun


In [79]:
# drop unnecessary columns 
clean_Jun2020_df = Jun2020_df.drop(columns = ["birth year", "gender","time"])
clean_Jun2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,1062,2020-06-01 00:00:03.3720,2020-06-01 00:17:46.2080,3419,Douglass St & 4 Ave,40.679279,-73.981540,3419,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,2020-06-01,Jun
1,3810,2020-06-01 00:00:03.5530,2020-06-01 01:03:33.9360,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,2020-06-01,Jun
2,1017,2020-06-01 00:00:09.6140,2020-06-01 00:17:06.8330,389,Broadway & Berry St,40.710446,-73.965251,3562,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,2020-06-01,Jun
3,226,2020-06-01 00:00:12.1780,2020-06-01 00:03:58.8640,3255,8 Ave & W 31 St,40.750585,-73.994685,505,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,2020-06-01,Jun
4,1437,2020-06-01 00:00:21.2550,2020-06-01 00:24:18.9650,367,E 53 St & Lexington Ave,40.758281,-73.970694,497,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,2020-06-01,Jun


In [80]:
# calculate summary fields
# total trips per month
Trips_Jun = clean_Jun2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Jun = clean_Jun2020_df.loc[clean_Jun2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Jun = Subtrips_Jun/Trips_Jun*100
Custtrips_Jun = clean_Jun2020_df.loc[clean_Jun2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Jun = Custtrips_Jun/Trips_Jun*100
# calculate total stations ids
start_station_id_count_Jun = clean_Jun2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Jun=clean_Jun2020_df["bikeid"].nunique()
bike_id_count_Jun

15849

In [82]:
# create monthly summary
Jun_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Jun],
                     "Subscriber Trips Count":[Subtrips_Jun],
                     "% Subscriber Trips":[percent_subtrips_Jun],
                     "Customer Trips Count":[Custtrips_Jun],
                     "% Customer Trips":[percent_custtrips_Jun],
                     "Start Station Count":[start_station_id_count_Jun],
                     "Bike ID Count":[bike_id_count_Jun]})
Jun_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,1882273,1306688,69.420748,575585,30.579252,958,15849


In [83]:
# add month column to summary
Jun_2020_Summary['Month'] = clean_Jun2020_df['starttimemonth']
Jun_2020_Summary=Jun_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Jun_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Jun,1882273,1306688,69.420748,575585,30.579252,958,15849


# July Data

In [84]:
d = pd.to_datetime(Jul2020_df['starttime'])
Jul2020_df['date'] = d.dt.date
Jul2020_df['time'] = d.dt.time

In [85]:
# convert create month column and convert to month name
Jul2020_df['starttimemonth'] = pd.DatetimeIndex(Jul2020_df['date']).month
Jul2020_df['starttimemonth'] = Jul2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Jul2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,341,2020-07-01 00:00:01.3020,2020-07-01 00:05:42.5650,3463,E 16 St & Irving Pl,40.735367,-73.987974,2003,1 Ave & E 18 St,40.733812,-73.980544,33861,Subscriber,1986,1,2020-07-01,00:00:01.302000,Jul
1,321,2020-07-01 00:00:01.3620,2020-07-01 00:05:22.9490,3463,E 16 St & Irving Pl,40.735367,-73.987974,2003,1 Ave & E 18 St,40.733812,-73.980544,31233,Subscriber,1990,2,2020-07-01,00:00:01.362000,Jul
2,2710,2020-07-01 00:00:06.6290,2020-07-01 00:45:17.3410,426,West St & Chambers St,40.717548,-74.013221,212,W 16 St & The High Line,40.743349,-74.006818,40329,Customer,1969,0,2020-07-01,00:00:06.629000,Jul
3,2685,2020-07-01 00:00:13.0220,2020-07-01 00:44:58.8640,426,West St & Chambers St,40.717548,-74.013221,212,W 16 St & The High Line,40.743349,-74.006818,17567,Subscriber,1998,1,2020-07-01,00:00:13.022000,Jul
4,191,2020-07-01 00:00:13.4660,2020-07-01 00:03:24.6230,3615,44 Dr & 21 St,40.748000,-73.946093,3127,9 St & 44 Rd,40.749660,-73.952100,43421,Subscriber,1992,1,2020-07-01,00:00:13.466000,Jul


In [86]:
# drop unnecessary columns 
clean_Jul2020_df = Jul2020_df.drop(columns = ["birth year", "gender","time"])
clean_Jul2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,341,2020-07-01 00:00:01.3020,2020-07-01 00:05:42.5650,3463,E 16 St & Irving Pl,40.735367,-73.987974,2003,1 Ave & E 18 St,40.733812,-73.980544,33861,Subscriber,2020-07-01,Jul
1,321,2020-07-01 00:00:01.3620,2020-07-01 00:05:22.9490,3463,E 16 St & Irving Pl,40.735367,-73.987974,2003,1 Ave & E 18 St,40.733812,-73.980544,31233,Subscriber,2020-07-01,Jul
2,2710,2020-07-01 00:00:06.6290,2020-07-01 00:45:17.3410,426,West St & Chambers St,40.717548,-74.013221,212,W 16 St & The High Line,40.743349,-74.006818,40329,Customer,2020-07-01,Jul
3,2685,2020-07-01 00:00:13.0220,2020-07-01 00:44:58.8640,426,West St & Chambers St,40.717548,-74.013221,212,W 16 St & The High Line,40.743349,-74.006818,17567,Subscriber,2020-07-01,Jul
4,191,2020-07-01 00:00:13.4660,2020-07-01 00:03:24.6230,3615,44 Dr & 21 St,40.748000,-73.946093,3127,9 St & 44 Rd,40.749660,-73.952100,43421,Subscriber,2020-07-01,Jul


In [87]:
# calculate summary fields
# total trips per month
Trips_Jul = clean_Jul2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Jul = clean_Jul2020_df.loc[clean_Jul2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Jul = Subtrips_Jul/Trips_Jul*100
Custtrips_Jul = clean_Jul2020_df.loc[clean_Jul2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Jul = Custtrips_Jul/Trips_Jul*100
# calculate total stations ids
start_station_id_count_Jul = clean_Jul2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Jul=clean_Jul2020_df["bikeid"].nunique()
bike_id_count_Jul

17075

In [88]:
# create monthly summary
Jul_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Jul],
                     "Subscriber Trips Count":[Subtrips_Jul],
                     "% Subscriber Trips":[percent_subtrips_Jul],
                     "Customer Trips Count":[Custtrips_Jul],
                     "% Customer Trips":[percent_custtrips_Jul],
                     "Start Station Count":[start_station_id_count_Jul],
                     "Bike ID Count":[bike_id_count_Jul]})
Jul_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,2105808,1530982,72.70283,574826,27.29717,989,17075


In [89]:
# add month column to summary
Jul_2020_Summary['Month'] = clean_Jul2020_df['starttimemonth']
Jul_2020_Summary=Jul_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Jul_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Jul,2105808,1530982,72.70283,574826,27.29717,989,17075


# Aug Data

In [90]:
d = pd.to_datetime(Aug2020_df['starttime'])
Aug2020_df['date'] = d.dt.date
Aug2020_df['time'] = d.dt.time

In [91]:
# convert create month column and convert to month name
Aug2020_df['starttimemonth'] = pd.DatetimeIndex(Aug2020_df['date']).month
Aug2020_df['starttimemonth'] = Aug2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Aug2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,2013,2020-08-01 00:00:00.8790,2020-08-01 00:33:34.7260,3295,Central Park W & W 96 St,40.791270,-73.964839,3992,W 169 St & Fort Washington Ave,40.842842,-73.942125,19706,Customer,1998,1,2020-08-01,00:00:00.879000,Aug
1,7002,2020-08-01 00:00:01.5910,2020-08-01 01:56:44.0720,3144,E 81 St & Park Ave,40.776777,-73.959010,3671,E 81 St & 2 Ave,40.774779,-73.954275,37191,Subscriber,1966,1,2020-08-01,00:00:01.591000,Aug
2,155,2020-08-01 00:00:03.5810,2020-08-01 00:02:38.6340,387,Centre St & Chambers St,40.712733,-74.004607,387,Centre St & Chambers St,40.712733,-74.004607,42561,Customer,1969,0,2020-08-01,00:00:03.581000,Aug
3,2720,2020-08-01 00:00:05.0130,2020-08-01 00:45:25.3270,3080,S 4 St & Rodney St,40.709340,-73.956080,3364,Carroll St & 5 Ave,40.675162,-73.981483,41607,Subscriber,1990,1,2020-08-01,00:00:05.013000,Aug
4,2938,2020-08-01 00:00:05.9420,2020-08-01 00:49:03.9640,3510,Adam Clayton Powell Blvd & W 123 St,40.807832,-73.949373,3282,5 Ave & E 88 St,40.783070,-73.959390,36366,Customer,1969,0,2020-08-01,00:00:05.942000,Aug


In [92]:
# drop unnecessary columns 
clean_Aug2020_df = Aug2020_df.drop(columns = ["birth year", "gender","time"])
clean_Aug2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,2013,2020-08-01 00:00:00.8790,2020-08-01 00:33:34.7260,3295,Central Park W & W 96 St,40.791270,-73.964839,3992,W 169 St & Fort Washington Ave,40.842842,-73.942125,19706,Customer,2020-08-01,Aug
1,7002,2020-08-01 00:00:01.5910,2020-08-01 01:56:44.0720,3144,E 81 St & Park Ave,40.776777,-73.959010,3671,E 81 St & 2 Ave,40.774779,-73.954275,37191,Subscriber,2020-08-01,Aug
2,155,2020-08-01 00:00:03.5810,2020-08-01 00:02:38.6340,387,Centre St & Chambers St,40.712733,-74.004607,387,Centre St & Chambers St,40.712733,-74.004607,42561,Customer,2020-08-01,Aug
3,2720,2020-08-01 00:00:05.0130,2020-08-01 00:45:25.3270,3080,S 4 St & Rodney St,40.709340,-73.956080,3364,Carroll St & 5 Ave,40.675162,-73.981483,41607,Subscriber,2020-08-01,Aug
4,2938,2020-08-01 00:00:05.9420,2020-08-01 00:49:03.9640,3510,Adam Clayton Powell Blvd & W 123 St,40.807832,-73.949373,3282,5 Ave & E 88 St,40.783070,-73.959390,36366,Customer,2020-08-01,Aug


In [100]:
# calculate summary fields
# total trips per month
Trips_Aug = clean_Aug2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Aug = clean_Aug2020_df.loc[clean_Aug2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Aug = Subtrips_Aug/Trips_Aug*100
Custtrips_Aug = clean_Aug2020_df.loc[clean_Aug2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Aug = Custtrips_Aug/Trips_Aug*100
# calculate total stations ids
start_station_id_count_Aug = clean_Aug2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Aug=clean_Aug2020_df["bikeid"].nunique()
bike_id_count_Aug

17835

In [94]:
# create monthly summary
Aug_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Aug],
                     "Subscriber Trips Count":[Subtrips_Aug],
                     "% Subscriber Trips":[percent_subtrips_Aug],
                     "Customer Trips Count":[Custtrips_Aug],
                     "% Customer Trips":[percent_custtrips_Aug],
                     "Start Station Count":[start_station_id_count_Aug],
                     "Bike ID Count":[bike_id_count_Aug]})
Aug_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,2329514,1671914,71.770936,657600,31.227918,1039,17835


In [95]:
# add month column to summary
Aug_2020_Summary['Month'] = clean_Aug2020_df['starttimemonth']
Aug_2020_Summary=Aug_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Aug_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Aug,2329514,1671914,71.770936,657600,31.227918,1039,17835


# Sept Data

In [96]:
d = pd.to_datetime(Sept2020_df['starttime'])
Sept2020_df['date'] = d.dt.date
Sept2020_df['time'] = d.dt.time

In [97]:
# convert create month column and convert to month name
Sept2020_df['starttimemonth'] = pd.DatetimeIndex(Sept2020_df['date']).month
Sept2020_df['starttimemonth'] = Sept2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Sept2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,4225,2020-09-01 00:00:01.0430,2020-09-01 01:10:26.6350,3508,St Nicholas Ave & Manhattan Ave,40.809725,-73.953149,116,W 17 St & 8 Ave,40.741776,-74.001497,44317,Customer,1979,1,2020-09-01,00:00:01.043000,Sep
1,1868,2020-09-01 00:00:04.8320,2020-09-01 00:31:13.7650,3621,27 Ave & 9 St,40.773983,-73.930913,3094,Graham Ave & Withers St,40.716981,-73.944859,37793,Customer,1991,1,2020-09-01,00:00:04.832000,Sep
2,1097,2020-09-01 00:00:06.8990,2020-09-01 00:18:24.2260,3492,E 118 St & Park Ave,40.800539,-73.941995,3959,Edgecombe Ave & W 145 St,40.823498,-73.943860,41438,Subscriber,1984,1,2020-09-01,00:00:06.899000,Sep
3,1473,2020-09-01 00:00:07.7440,2020-09-01 00:24:41.1800,3946,St Nicholas Ave & W 137 St,40.818477,-73.947568,4002,W 144 St & Adam Clayton Powell Blvd,40.820877,-73.939249,35860,Customer,1990,2,2020-09-01,00:00:07.744000,Sep
4,1193,2020-09-01 00:00:12.2020,2020-09-01 00:20:05.5470,3081,Graham Ave & Grand St,40.711863,-73.944024,3048,Putnam Ave & Nostrand Ave,40.684020,-73.949770,26396,Customer,1969,0,2020-09-01,00:00:12.202000,Sep


In [98]:
# drop unnecessary columns 
clean_Sept2020_df = Sept2020_df.drop(columns = ["birth year", "gender","time"])
clean_Sept2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,4225,2020-09-01 00:00:01.0430,2020-09-01 01:10:26.6350,3508,St Nicholas Ave & Manhattan Ave,40.809725,-73.953149,116,W 17 St & 8 Ave,40.741776,-74.001497,44317,Customer,2020-09-01,Sep
1,1868,2020-09-01 00:00:04.8320,2020-09-01 00:31:13.7650,3621,27 Ave & 9 St,40.773983,-73.930913,3094,Graham Ave & Withers St,40.716981,-73.944859,37793,Customer,2020-09-01,Sep
2,1097,2020-09-01 00:00:06.8990,2020-09-01 00:18:24.2260,3492,E 118 St & Park Ave,40.800539,-73.941995,3959,Edgecombe Ave & W 145 St,40.823498,-73.943860,41438,Subscriber,2020-09-01,Sep
3,1473,2020-09-01 00:00:07.7440,2020-09-01 00:24:41.1800,3946,St Nicholas Ave & W 137 St,40.818477,-73.947568,4002,W 144 St & Adam Clayton Powell Blvd,40.820877,-73.939249,35860,Customer,2020-09-01,Sep
4,1193,2020-09-01 00:00:12.2020,2020-09-01 00:20:05.5470,3081,Graham Ave & Grand St,40.711863,-73.944024,3048,Putnam Ave & Nostrand Ave,40.684020,-73.949770,26396,Customer,2020-09-01,Sep


In [99]:
# calculate summary fields
# total trips per month
Trips_Sept = clean_Sept2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Sept = clean_Sept2020_df.loc[clean_Sept2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Sept = Subtrips_Sept/Trips_Sept*100
Custtrips_Sept = clean_Sept2020_df.loc[clean_Sept2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Sept = Custtrips_Sept/Trips_Sept*100
# calculate total stations ids
start_station_id_count_Sept = clean_Sept2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Sept=clean_Sept2020_df["bikeid"].nunique()
bike_id_count_Sept

18283

In [101]:
# create monthly summary
Sept_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Sept],
                     "Subscriber Trips Count":[Subtrips_Sept],
                     "% Subscriber Trips":[percent_subtrips_Sept],
                     "Customer Trips Count":[Custtrips_Sept],
                     "% Customer Trips":[percent_custtrips_Sept],
                     "Start Station Count":[start_station_id_count_Sept],
                     "Bike ID Count":[bike_id_count_Sept]})
Sept_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,2488225,1817674,73.05103,670551,26.94897,1087,18283


In [102]:
# add month column to summary
Sept_2020_Summary['Month'] = clean_Sept2020_df['starttimemonth']
Sept_2020_Summary=Sept_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Sept_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Sep,2488225,1817674,73.05103,670551,26.94897,1087,18283


# Oct Data

In [103]:
d = pd.to_datetime(Oct2020_df['starttime'])
Oct2020_df['date'] = d.dt.date
Oct2020_df['time'] = d.dt.time

In [104]:
# convert create month column and convert to month name
Oct2020_df['starttimemonth'] = pd.DatetimeIndex(Oct2020_df['date']).month
Oct2020_df['starttimemonth'] = Oct2020_df['starttimemonth'].apply(lambda x: calendar.month_abbr[x])
Oct2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,time,starttimemonth
0,1403,2020-10-01 00:00:10.7870,2020-10-01 00:23:34.5400,224,Spruce St & Nassau St,40.711464,-74.005524,3347,Van Brunt St & Wolcott St,40.677343,-74.012751,39079,Subscriber,1995,1,2020-10-01,00:00:10.787000,Oct
1,462,2020-10-01 00:00:18.2390,2020-10-01 00:08:00.5710,3161,W 76 St & Columbus Ave,40.780184,-73.977285,3158,W 63 St & Broadway,40.771639,-73.982614,41267,Subscriber,1998,2,2020-10-01,00:00:18.239000,Oct
2,770,2020-10-01 00:00:21.3070,2020-10-01 00:13:12.0840,354,Emerson Pl & Myrtle Ave,40.693631,-73.962236,3761,Cedar St & Myrtle Ave,40.697842,-73.926241,47398,Customer,1996,1,2020-10-01,00:00:21.307000,Oct
3,233,2020-10-01 00:00:25.0080,2020-10-01 00:04:18.6410,3141,1 Ave & E 68 St,40.765005,-73.958185,464,E 56 St & 3 Ave,40.759345,-73.967597,36482,Subscriber,1993,1,2020-10-01,00:00:25.008000,Oct
4,126,2020-10-01 00:00:25.3100,2020-10-01 00:02:32.1330,335,Washington Pl & Broadway,40.729039,-73.994046,229,Great Jones St,40.727434,-73.993790,45319,Subscriber,1989,1,2020-10-01,00:00:25.310000,Oct


In [105]:
# drop unnecessary columns 
clean_Oct2020_df = Oct2020_df.drop(columns = ["birth year", "gender","time"])
clean_Oct2020_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,1403,2020-10-01 00:00:10.7870,2020-10-01 00:23:34.5400,224,Spruce St & Nassau St,40.711464,-74.005524,3347,Van Brunt St & Wolcott St,40.677343,-74.012751,39079,Subscriber,2020-10-01,Oct
1,462,2020-10-01 00:00:18.2390,2020-10-01 00:08:00.5710,3161,W 76 St & Columbus Ave,40.780184,-73.977285,3158,W 63 St & Broadway,40.771639,-73.982614,41267,Subscriber,2020-10-01,Oct
2,770,2020-10-01 00:00:21.3070,2020-10-01 00:13:12.0840,354,Emerson Pl & Myrtle Ave,40.693631,-73.962236,3761,Cedar St & Myrtle Ave,40.697842,-73.926241,47398,Customer,2020-10-01,Oct
3,233,2020-10-01 00:00:25.0080,2020-10-01 00:04:18.6410,3141,1 Ave & E 68 St,40.765005,-73.958185,464,E 56 St & 3 Ave,40.759345,-73.967597,36482,Subscriber,2020-10-01,Oct
4,126,2020-10-01 00:00:25.3100,2020-10-01 00:02:32.1330,335,Washington Pl & Broadway,40.729039,-73.994046,229,Great Jones St,40.727434,-73.993790,45319,Subscriber,2020-10-01,Oct


In [106]:
# calculate summary fields
# total trips per month
Trips_Oct = clean_Oct2020_df["tripduration"].count()
# Calculate total trips by subscriber and customer and percentage of each
Subtrips_Oct = clean_Oct2020_df.loc[clean_Oct2020_df.usertype == "Subscriber", "usertype"].count()
percent_subtrips_Oct = Subtrips_Oct/Trips_Oct*100
Custtrips_Oct = clean_Oct2020_df.loc[clean_Oct2020_df.usertype == "Customer", "usertype"].count()
percent_custtrips_Oct = Custtrips_Oct/Trips_Oct*100
# calculate total stations ids
start_station_id_count_Oct = clean_Oct2020_df["start station id"].nunique()
# Calculate total bike ids
bike_id_count_Oct=clean_Oct2020_df["bikeid"].nunique()
bike_id_count_Oct

18472

In [107]:
# create monthly summary
Oct_2020_Summary = pd.DataFrame({
                     "Total Trips":[Trips_Oct],
                     "Subscriber Trips Count":[Subtrips_Oct],
                     "% Subscriber Trips":[percent_subtrips_Oct],
                     "Customer Trips Count":[Custtrips_Oct],
                     "% Customer Trips":[percent_custtrips_Oct],
                     "Start Station Count":[start_station_id_count_Oct],
                     "Bike ID Count":[bike_id_count_Oct]})
Oct_2020_Summary

,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,2248869,1702115,75.687601,546754,24.312399,1144,18472


In [109]:
# add month column to summary
Oct_2020_Summary['Month'] = clean_Oct2020_df['starttimemonth']
Oct_2020_Summary=Oct_2020_Summary[['Month','Total Trips','Subscriber Trips Count','% Subscriber Trips',
                                   'Customer Trips Count','% Customer Trips','Start Station Count','Bike ID Count',]]
Oct_2020_Summary

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Oct,2248869,1702115,75.687601,546754,24.312399,1144,18472


# Start and End station Oct Data
For Maps

In [111]:
# create start station summary
startStation_Oct2020_df=clean_Oct2020_df.groupby(["start station id",'start station latitude',
                                                  'start station longitude','start station name']).count().reset_index()
startStation_Oct2020_final = startStation_Oct2020_df.rename(columns={"tripduration":"count"})
startStation_Oct2020_final.head()

,start station id,start station latitude,start station longitude,start station name,count,starttime,stoptime,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,date,starttimemonth
0,72,40.767272,-73.993929,W 52 St & 11 Ave,3738,3738,3738,3738,3738,3738,3738,3738,3738,3738,3738
1,79,40.719116,-74.006667,Franklin St & W Broadway,2396,2396,2396,2396,2396,2396,2396,2396,2396,2396,2396
2,82,40.711174,-74.000165,St James Pl & Pearl St,1283,1283,1283,1283,1283,1283,1283,1283,1283,1283,1283
3,83,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,1965,1965,1965,1965,1965,1965,1965,1965,1965,1965,1965
4,116,40.741776,-74.001497,W 17 St & 8 Ave,5059,5059,5059,5059,5059,5059,5059,5059,5059,5059,5059


In [115]:
# clean start station summary
CleanStartStation_Oct2020_df = startStation_Oct2020_final.drop(columns=["starttime","stoptime",
                                      "end station id","end station name","end station latitude","end station longitude",
                                      "bikeid", "usertype", "date", "starttimemonth"])
CleanStartStation_Oct2020_df.head()

,start station id,start station latitude,start station longitude,start station name,count
0,72,40.767272,-73.993929,W 52 St & 11 Ave,3738
1,79,40.719116,-74.006667,Franklin St & W Broadway,2396
2,82,40.711174,-74.000165,St James Pl & Pearl St,1283
3,83,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,1965
4,116,40.741776,-74.001497,W 17 St & 8 Ave,5059


In [112]:
# create end station summary
endStation_Oct2020_df=clean_Oct2020_df.groupby(["end station id",'end station latitude',
                                                'end station longitude','end station name']).count().reset_index()
endStation_Oct2020_final = endStation_Oct2020_df.rename(columns={"tripduration":"count"})
endStation_Oct2020_final.head()

,end station id,end station latitude,end station longitude,end station name,count,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,bikeid,usertype,date,starttimemonth
0,72,40.767272,-73.993929,W 52 St & 11 Ave,3789,3789,3789,3789,3789,3789,3789,3789,3789,3789,3789
1,79,40.719116,-74.006667,Franklin St & W Broadway,2460,2460,2460,2460,2460,2460,2460,2460,2460,2460,2460
2,82,40.711174,-74.000165,St James Pl & Pearl St,1292,1292,1292,1292,1292,1292,1292,1292,1292,1292,1292
3,83,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,1980,1980,1980,1980,1980,1980,1980,1980,1980,1980,1980
4,116,40.741776,-74.001497,W 17 St & 8 Ave,5077,5077,5077,5077,5077,5077,5077,5077,5077,5077,5077


In [116]:
# clean end station summary
CleanEndStation_Oct2020_df = endStation_Oct2020_final.drop(columns=["starttime","stoptime",
                                      "start station id","start station name","start station latitude","start station longitude",
                                      "bikeid", "usertype", "date", "starttimemonth"])
CleanEndStation_Oct2020_df.head()

,end station id,end station latitude,end station longitude,end station name,count
0,72,40.767272,-73.993929,W 52 St & 11 Ave,3789
1,79,40.719116,-74.006667,Franklin St & W Broadway,2460
2,82,40.711174,-74.000165,St James Pl & Pearl St,1292
3,83,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,1980
4,116,40.741776,-74.001497,W 17 St & 8 Ave,5077


# Create Annual Summary and Export Data

In [117]:
# concat monthly summaries into annual summary
Annual_Summary = pd.concat([Jan_2020_Summary,Feb_2020_Summary,Mar_2020_Summary,Apr_2020_Summary, 
                            May_2020_Summary,Jun_2020_Summary, Jul_2020_Summary, Aug_2020_Summary, Sept_2020_Summary, 
                            Oct_2020_Summary], ignore_index=True)
Annual_Summary 

,Month,Total Trips,Subscriber Trips Count,% Subscriber Trips,Customer Trips Count,% Customer Trips,Start Station Count,Bike ID Count
0,Jan,1240596,1145819,92.360366,94777,7.639634,893,13049
1,Feb,1146830,1054927,91.986345,91903,8.013655,890,13218
2,Mar,1068457,914068,85.550284,154389,14.449716,889,14882
3,Apr,682762,516495,75.647883,166267,24.352117,887,14780
4,May,1487890,1012059,68.019746,475831,31.980254,923,15003
5,Jun,1882273,1306688,69.420748,575585,30.579252,958,15849
6,Jul,2105808,1530982,72.702830,574826,27.297170,989,17075
7,Aug,2329514,1671914,71.770936,657600,31.227918,1039,17835
8,Sep,2488225,1817674,73.051030,670551,26.948970,1087,18283
9,Oct,2248869,1702115,75.687601,546754,24.312399,1144,18472


In [118]:
# export 2020 Annual Summary to csv
Annual_Summary.to_csv("2020_Annual_Summary.csv", index=False)

In [119]:
#export start point info
CleanStartStation_Oct2020_df.to_csv("Oct_2020_Start_Station_info.csv", index=False)

# export end point info
CleanEndStation_Oct2020_df.to_csv("Oct_2020_end_Station_info.csv", index=False)